In [13]:
import cv2 
import os  
import sys 
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import tensorflow as tf 

In [14]:
from tensorflow.keras.models import Sequential,Model, load_model 
from tensorflow.keras.layers import Activation, Dense, Input, Dropout, Flatten 
from tensorflow.keras.optimizers import Adam, SGD, RMSprop 
from tensorflow.keras.datasets import mnist 

In [15]:
def get_show_shape(images):
    for i in range(len(images)):
        print(f"shape {images[i].shape}")

def get_show_images(images, labels, ncols=15):
    fig, axes = plt.subplots(3, 5)
    fig.set_size_inches(8, 5)
    for i in range(ncols):
        ax = axes[i//5, i%5]
        ax.imshow(images[i], cmap="gray")
        ax.axis("off")
        ax.set_title(str(labels[i]))
    plt.tight_layout()
    plt.show()

In [16]:
def get_preprocessed_scaler(x_train, x_test):
    x_train = x_train / x_train.max()
    x_test = x_test / x_test.max()

    return x_train, x_test 

In [27]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()
# get_show_shape([x_train, y_train, x_test, y_test])
# get_show_images(x_train[0:15], y_train[0:15], ncols=15)

In [ ]:
x_train = x_train / x_train.max()
x_test = x_test / x_test.max()

#### layer initializer  
- 초기화 함수는 keras layer의 parameter의 초기값을 어떤 방식을 생성할지 결정한다. 
- 모든 층에서 다 똑같지는 않지만 대부분의 경우 kernel_initializer와 bias_initializer 인자를 사용해서 weight와 bias 초기화 함수를 지정한다.
- keras.initializers.Initializer() 클래스를 상속받는다. 

In [30]:
dense = Dense(256, activation="relu")
dense.get_config()["kernel_initializer"]


{'class_name': 'GlorotUniform', 'config': {'seed': None}}

In [31]:
dense = Dense(256, kernel_initializer="he_normal", activation="relu")
print(dense.get_config()["kernel_initializer"])

{'class_name': 'HeNormal', 'config': {'seed': None}}


#### Regularization
- 모델의 과대적합을 해소하기 위해서 L1, L2 규제를 적용하기도 한다. 
- keras layer는 기본값으로 규제를 적용하고 있지 않다. 따라서 규제를 적용할 경우에는 별도로 설정이 필요하다. 
- keras layer의 규제를 적용하기 위해서는 kernel_regularizer에 규제를 지정한다.  

In [32]:
dense = Dense(256, kernel_regularizer="l1", activation="relu")
dense.get_config()["kernel_regularizer"]

{'class_name': 'L1', 'config': {'l1': 0.009999999776482582}}

#### Dropout 
- 딥러닝 모델의 층이 넓고 깊어질 때 모델은 훈련에 주어진 샘플에 과하게 적합하도록 학습하는 경향이 있다. 
- 훈련데이터셋에 너무 적응하여 검증 데이터셋이나 테스트데이터셋에 일반화된 성능을 갖지 못하는문제가 발생하기 때문에 Dropout는 과대적합 문제를 해결하기 위해서 제안된 것이다. 
- Dropout에 입력되는 숫자는 노드에서 제거되는 비율을 나타낸다.  

In [33]:
Dropout(0.25)

#### BatchNormalization
* BatchNormalization는 각 층에서 활성화 함수를 통과하기 전에 mini-bacth의 스케일을 정규화한다. 
* 다음층으로 데이터가 전달되기 전에 스케일을 조정하기 때문에 보다 안정적인 훈련이 가능하고, 성능이 향상된다. 
  